# Import Packages

In [2]:
import sys
import os
import xarray as xr
import numpy
import time
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime
import random
sys.path.append(os.path.join(os.path.abspath(''), '..', 'modules'))

from dask.diagnostics import ProgressBar
import lorenz
import statlorenz
import plotting
import general

# Set Random Seed

In [2]:
random.seed(datetime.now())

# Metadata

In [3]:
# General parameters
## Hindcast parameters
Ncores                = 20     # Number of cores used for creating the experiments
Nens                  = 100    # Number of ensemble members for each forecast
Nexp                  = 100    # Number of years
length_forecast       = 20     # Length of the forecast in timeunits

## Model parameters for reference run
rho                   = 28
beta                  = 8./3.
sigma                 = 10.0

## Model parameters for forecast run
rho_hin               = 28
beta_hin              = 8./3.
sigma_hin             = 10.0

## Observational spread
std_obs               = 0.01

# Experiment specific parameters
## Incremental spread experiment
Nsample_increment     = 501    # Number of samples
std_ens_min           = 0.001
std_ens_max           = 0.1

## Reliable Spread Experiment
std_ens_min_reliable = 0.01
std_ens_max_reliable = 0.01
Nsample_reliable     = 100

## Low Spread Experiment
std_ens_min_over = 0.001
std_ens_max_over = 0.001
Nsample_over     = 100

## High Spread Experiment
std_ens_min_under = 0.1
std_ens_max_under = 0.1
Nsample_under     = 100

# Define Output Paths

In [4]:
path_data    = os.path.join(os.path.abspath(""),"..","..","data")
path_results = os.path.join(os.path.abspath(""),"..","..","results")

In [5]:
path_numbers = os.path.join(path_results,"numbers")
path_pool = os.path.join(path_data,"InitialConditions")

In [6]:
raw_output_path_incremental = os.path.join(path_data, "PerfectModel_Incremental")
raw_output_path_under       = os.path.join(path_data, "PerfectModel_Under") 
raw_output_path_over        = os.path.join(path_data, "PerfectModel_Over")
raw_output_path_reliable    = os.path.join(path_data, "PerfectModel_Reliable")

# Create Output Folders

In [106]:
os.system("mkdir " + path_numbers)
os.system("mkdir " + raw_output_path_incremental)
os.system("mkdir " + raw_output_path_under)
os.system("mkdir " + raw_output_path_over)
os.system("mkdir " + raw_output_path_reliable)

os.system("mkdir "+ path_pool)

os.system("mkdir " + os.path.join(path_results,"Incremental"))
os.system("mkdir " + os.path.join(path_results,"Overconfident"))
os.system("mkdir " + os.path.join(path_results,"Underconfident"))
os.system("mkdir " + os.path.join(path_results,"Reliable"))

os.system("mkdir " + path_results)
os.system("mkdir " + path_numbers)

256

# Execution

## Create Pool of Initial Conditions for Experiment

In [8]:
filename_pool = os.path.join(path_pool,"Initial_conditions.nc")
if os.path.exists(filename_pool):
    pool_of_initial_conditions=xr.load_dataarray(filename_pool)
    print("Pool Of Initial conditions already exists. Pool loaded")
else:
    pool_of_initial_conditions=lorenz.Create_Pool_Of_Initial_Conditions()
    pool_of_initial_conditions.to_netcdf(filename_pool)

Pool Of Initial conditions already exists. Pool loaded


## Create Permutations

In [24]:
path_permutations=os.path.join(path_data,"Permutations")
os.system("mkdir "+ path_permutations)
filename_permutations = os.path.join(path_permutations,"Permutations.nc")
if os.path.exists(filename_permutations):
    print("Pool Of Initial conditions already exists. ")
    permutations = xr.load_dataarray(filename_permutations)
else:
    print("Permutations didnt exist. Create new one")
    permutations=statlorenz.create_random_permutations(Nens=Nens)
    permutations.to_netcdf(filename_permutations)

Pool Of Initial conditions already exists. 


## Run Experiments

### Run Incremental Experiments over range of different std_ens_x from 0.001 to 0.1

In [17]:
string_incremental = "".join(["python CreateExperiments.py",
         " --outfolder="     , raw_output_path_incremental,
         " --Ncores="        , str(Ncores),
                        " --Nens="          , str(Nens),            
                        " --Nexp="          , str(Nexp),
                        " --rho="           , str(rho),
                        " --sigma="         , str(sigma),
                        " --beta="          , str(beta),
                        " --rho_hin="       , str(rho_hin),
                        " --sigma_hin="     , str(sigma_hin),
                        " --beta_hin="      , str(beta_hin),
                        " --std_obs="       , str(std_obs),
                        " --std_ens_min="   , str(std_ens_min),
                        " --std_ens_max="   , str(std_ens_max),
                        " --Nsample="       , str(Nsample_increment),
                        " --length_forecast=", str(length_forecast)])

'python CreateExperiments.py --outfolder=/mnt/lustre02/work/uo1075/u301101/SNP_Lorenz_clean/src/scripts/../../data/PerfectModel_Incremental --Ncores=20 --Nens=100 --Nexp=100 --rho=28 --sigma=10.0 --beta=2.6666666666666665 --rho_hin=28 --sigma_hin=10.0 --beta_hin=2.6666666666666665 --std_obs=0.01 --std_ens_min=0.001 --std_ens_max=0.1 --Nsample=501 --length_forecast=20'

In [ ]:
! {string_incremental}

### Run 100 Reliable Experiments for std_ens = 0.01 = std_obs

In [7]:
string_reliable ="".join(["python CreateExperiments.py",
                        " --outfolder="     , raw_output_path_reliable,
                        " --Ncores="        , str(Ncores),
                        " --Nens="          , str(Nens),            
                        " --Nexp="          , str(Nexp),
                        " --rho="           , str(rho),
                        " --sigma="         , str(sigma),
                        " --beta="          , str(beta),
                        " --rho_hin="       , str(rho_hin),
                        " --sigma_hin="     , str(sigma_hin),
                        " --beta_hin="      , str(beta_hin),
                        " --std_obs="       , str(std_obs),
                        " --std_ens_min="   , str(std_ens_min_reliable),
                        " --std_ens_max="   , str(std_ens_max_reliable),
                        " --Nsample="       , str(Nsample_reliable),
                        " --length_forecast=", str(length_forecast)])

string_reliable

'python CreateExperiments.py --outfolder=/mnt/lustre02/work/uo1075/u301101/SNP_Lorenz_clean/src/scripts/../../data/PerfectModel_Reliable --Ncores=20 --Nens=100 --Nexp=100 --rho=28 --sigma=10.0 --beta=2.6666666666666665 --rho_hin=28 --sigma_hin=10.0 --beta_hin=2.6666666666666665 --std_obs=0.01 --std_ens_min=0.01 --std_ens_max=0.01 --Nsample=100 --length_forecast=20'

In [8]:
! {string_reliable}

New Set of Sample started with the following parameters: 
---------------------------------------------------------
output_folder: /mnt/lustre02/work/uo1075/u301101/SNP_Lorenz_clean/src/scripts/../../data/PerfectModel_Reliable
Nens: 100
Nexp: 100
length_forecast: 20
rho: 28
beta: 2.6666666666666665
sigma: 10.0
rho_hin: 28
beta_hin: 2.6666666666666665
sigma_hin: 10.0
std_obs: 0.01
std_ens_min: 0.01
std_ens_max: 0.01
Nsample: 100
Ncores: 20
out_timestep: 0.01
Pool of Initial conditions loaded !
the different samples have the following standard deviations:
[0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01
 0.01 0.01 0.0

### Run 100 Oveconfident Experiments for std_ens = 0.001 and std_obs =0.01

In [9]:
string_over ="".join(["python CreateExperiments.py",
                        " --outfolder="     , raw_output_path_over,
                        " --Ncores="        , str(Ncores),
                        " --Nens="          , str(Nens),            
                        " --Nexp="          , str(Nexp),
                        " --rho="           , str(rho),
                        " --sigma="         , str(sigma),
                        " --beta="          , str(beta),
                        " --rho_hin="       , str(rho_hin),
                        " --sigma_hin="     , str(sigma_hin),
                        " --beta_hin="      , str(beta_hin),
                        " --std_obs="       , str(std_obs),
                        " --std_ens_min="   , str(std_ens_min_over),
                        " --std_ens_max="   , str(std_ens_max_over),
                        " --Nsample="       , str(Nsample_over),
                        " --length_forecast=", str(length_forecast)])

In [10]:
! {string_over}

New Set of Sample started with the following parameters: 
---------------------------------------------------------
output_folder: /mnt/lustre02/work/uo1075/u301101/SNP_Lorenz_clean/src/scripts/../../data/PerfectModel_Over
Nens: 100
Nexp: 100
length_forecast: 20
rho: 28
beta: 2.6666666666666665
sigma: 10.0
rho_hin: 28
beta_hin: 2.6666666666666665
sigma_hin: 10.0
std_obs: 0.01
std_ens_min: 0.001
std_ens_max: 0.001
Nsample: 100
Ncores: 20
out_timestep: 0.01
Pool of Initial conditions loaded !
the different samples have the following standard deviations:
[0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001
 0.0

## Run 100 Underconfident Experiments for std_ens= 0.1 and std_obs=0.01

In [11]:
string_under ="".join(["python CreateExperiments.py",
                        " --outfolder="     , raw_output_path_under,
                        " --Ncores="        , str(Ncores),
                        " --Nens="          , str(Nens),            
                        " --Nexp="          , str(Nexp),
                        " --rho="           , str(rho),
                        " --sigma="         , str(sigma),
                        " --beta="          , str(beta),
                        " --rho_hin="       , str(rho_hin),
                        " --sigma_hin="     , str(sigma_hin),
                        " --beta_hin="      , str(beta_hin),
                        " --std_obs="       , str(std_obs),
                        " --std_ens_min="   , str(std_ens_min_under),
                        " --std_ens_max="   , str(std_ens_max_under),
                        " --Nsample="       , str(Nsample_under),
                        " --length_forecast=", str(length_forecast)])
string_under

'python CreateExperiments.py --outfolder=/mnt/lustre02/work/uo1075/u301101/SNP_Lorenz_clean/src/scripts/../../data/PerfectModel_Under --Ncores=20 --Nens=100 --Nexp=100 --rho=28 --sigma=10.0 --beta=2.6666666666666665 --rho_hin=28 --sigma_hin=10.0 --beta_hin=2.6666666666666665 --std_obs=0.01 --std_ens_min=0.1 --std_ens_max=0.1 --Nsample=100 --length_forecast=20'

In [12]:
! {string_under}

New Set of Sample started with the following parameters: 
---------------------------------------------------------
output_folder: /mnt/lustre02/work/uo1075/u301101/SNP_Lorenz_clean/src/scripts/../../data/PerfectModel_Under
Nens: 100
Nexp: 100
length_forecast: 20
rho: 28
beta: 2.6666666666666665
sigma: 10.0
rho_hin: 28
beta_hin: 2.6666666666666665
sigma_hin: 10.0
std_obs: 0.01
std_ens_min: 0.1
std_ens_max: 0.1
Nsample: 100
Ncores: 20
out_timestep: 0.01
Pool of Initial conditions loaded !
the different samples have the following standard deviations:
[0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1
 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1 0.1]
Parameters setup done!
Experiment 0 st

## Run postprocessing

### Create The Folders where experiments are saved after postprocessing

In [11]:
timescale_list = ["dm","wm","mm","sm"]
for timescale in timescale_list:
    os.system("mkdir " + os.path.join(raw_output_path_incremental, timescale))
    os.system("mkdir " + os.path.join(raw_output_path_over, timescale))
    os.system("mkdir " + os.path.join(raw_output_path_under, timescale))
    os.system("mkdir " + os.path.join(raw_output_path_reliable, timescale))

### Create Analysis by Adding Noise to teference run and calculate time windows

#### Incremental Spread

In [10]:
# Create Empty Lists to save the experiments    
dm=[]
wm=[]
mm=[]
sm=[]


for sample in np.arange(418,Nsample_increment):
    start = datetime.now()
    dataset = xr.load_dataset(os.path.join(raw_output_path_incremental, "Experiment_"+str(sample).zfill(3)+".nc"))
    
    # Add to the reference run gaussian noise and save it as the same file again with the additional analyis
    dataset = lorenz.Reference_To_Analysis(dataset)
    dataset.to_netcdf(os.path.join(raw_output_path_incremental, "Experiment_" + str(sample).zfill(3)+"_analysis.nc"))
    
    # Create daily, weekly, monthly and seasonal means fromt he dataset
    dm_tmp,wm_tmp,mm_tmp,sm_tmp = lorenz.Lorenz_Time_Windows(dataset)
        
    dm_tmp.to_netcdf(os.path.join(raw_output_path_incremental, "dm","dm_" + str(sample).zfill(3) + ".nc"))
    wm_tmp.to_netcdf(os.path.join(raw_output_path_incremental, "wm","wm_" + str(sample).zfill(3) + ".nc"))
    mm_tmp.to_netcdf(os.path.join(raw_output_path_incremental, "mm","mm_" + str(sample).zfill(3) + ".nc"))
    sm_tmp.to_netcdf(os.path.join(raw_output_path_incremental, "sm","sm_" + str(sample).zfill(3) + ".nc"))
        
    end = datetime.now()
    
    dm.append(dm_tmp)
    wm.append(wm_tmp)
    mm.append(mm_tmp)
    sm.append(sm_tmp)
    
    print("Experiment " + str(sample).zfill(3) + "/" + str(Nsample_increment) + " done in " + str((end-start).seconds) + " seconds!", end ="\r")

#### Overconfident forecasts

In [11]:
# Create Empty Lists to save the experiments    
dm_over=[]
wm_over=[]
mm_over=[]
sm_over=[]


for sample in np.arange(Nsample_over):
    start = datetime.now()
    dataset = xr.load_dataset(os.path.join(raw_output_path_over , "Experiment_"+str(sample).zfill(3)+".nc"))
    
    # Add to the reference run gaussian noise and save it as the same file again with the additional analyis
    dataset = lorenz.Reference_To_Analysis(dataset)
    dataset.to_netcdf(os.path.join(raw_output_path_over, "Experiment_" + str(sample).zfill(3)+"_analysis.nc"))
    
    # Create daily, weekly, monthly and seasonal means fromt he dataset
    dm_over_tmp,wm_over_tmp,mm_over_tmp,sm_over_tmp = lorenz.Lorenz_Time_Windows(dataset)
        
    dm_over_tmp.to_netcdf(os.path.join(raw_output_path_over, "dm","dm_" + str(sample).zfill(3) + ".nc"))
    wm_over_tmp.to_netcdf(os.path.join(raw_output_path_over, "wm","wm_" + str(sample).zfill(3) + ".nc"))
    mm_over_tmp.to_netcdf(os.path.join(raw_output_path_over, "mm","mm_" + str(sample).zfill(3) + ".nc"))
    sm_over_tmp.to_netcdf(os.path.join(raw_output_path_over, "sm","sm_" + str(sample).zfill(3) + ".nc"))
        
    end = datetime.now()
    
    dm_over.append(dm_over_tmp)
    wm_over.append(wm_over_tmp)
    mm_over.append(mm_over_tmp)
    sm_over.append(sm_over_tmp)
    
    print("Experiment " + str(sample).zfill(3) + "/" + str(Nsample_over) + " done in " + str((end-start).seconds) + " seconds!", end ="\r")

#### Underconfident forecasts

In [12]:
# Create Empty Lists to save the experiments    
dm_under=[]
wm_under=[]
mm_under=[]
sm_under=[]


for sample in np.arange(Nsample_under):
    start = datetime.now()
    dataset = xr.load_dataset(os.path.join(raw_output_path_under , "Experiment_"+str(sample).zfill(3)+".nc"))
    
    # Add to the reference run gaussian noise and save it as the same file again with the additional analyis
    dataset = lorenz.Reference_To_Analysis(dataset)
    dataset.to_netcdf(os.path.join(raw_output_path_under, "Experiment_" + str(sample).zfill(3)+"_analysis.nc"))
    
    # Create daily, weekly, monthly and seasonal means fromt he dataset
    dm_under_tmp,wm_under_tmp,mm_under_tmp,sm_under_tmp = lorenz.Lorenz_Time_Windows(dataset)
        
    dm_under_tmp.to_netcdf(os.path.join(raw_output_path_under, "dm","dm_" + str(sample).zfill(3) + ".nc"))
    wm_under_tmp.to_netcdf(os.path.join(raw_output_path_under, "wm","wm_" + str(sample).zfill(3) + ".nc"))
    mm_under_tmp.to_netcdf(os.path.join(raw_output_path_under, "mm","mm_" + str(sample).zfill(3) + ".nc"))
    sm_under_tmp.to_netcdf(os.path.join(raw_output_path_under, "sm","sm_" + str(sample).zfill(3) + ".nc"))
        
    end = datetime.now()
    
    dm_under.append(dm_under_tmp)
    wm_under.append(wm_under_tmp)
    mm_under.append(mm_under_tmp)
    sm_under.append(sm_under_tmp)
    
    print("Experiment " + str(sample).zfill(3) + "/" + str(Nsample_under) + " done in " + str((end-start).seconds) + " seconds!", end ="\r")

### Reliable Forecasts

In [12]:
# Create Empty Lists to save the experiments    
dm_reliable=[]
wm_reliable=[]
mm_reliable=[]
sm_reliable=[]


for sample in np.arange(Nsample_reliable):
    start = datetime.now()
    dataset = xr.load_dataset(os.path.join(raw_output_path_reliable , "Experiment_"+str(sample).zfill(3)+".nc"))
    
    # Add to the reference run gaussian noise and save it as the same file again with the additional analyis
    dataset = lorenz.Reference_To_Analysis(dataset)
    dataset.to_netcdf(os.path.join(raw_output_path_reliable, "Experiment_" + str(sample).zfill(3)+"_analysis.nc"))
    
    # Create daily, weekly, monthly and seasonal means fromt he dataset
    dm_reliable_tmp,wm_reliable_tmp,mm_reliable_tmp,sm_reliable_tmp = lorenz.Lorenz_Time_Windows(dataset)
        
    dm_reliable_tmp.to_netcdf(os.path.join(raw_output_path_reliable, "dm","dm_" + str(sample).zfill(3) + ".nc"))
    wm_reliable_tmp.to_netcdf(os.path.join(raw_output_path_reliable, "wm","wm_" + str(sample).zfill(3) + ".nc"))
    mm_reliable_tmp.to_netcdf(os.path.join(raw_output_path_reliable, "mm","mm_" + str(sample).zfill(3) + ".nc"))
    sm_reliable_tmp.to_netcdf(os.path.join(raw_output_path_reliable, "sm","sm_" + str(sample).zfill(3) + ".nc"))
        
    end = datetime.now()
    
    dm_reliable.append(dm_reliable_tmp)
    wm_reliable.append(wm_reliable_tmp)
    mm_reliable.append(mm_reliable_tmp)
    sm_reliable.append(sm_reliable_tmp)
    
    print("Experiment " + str(sample).zfill(3) + "/" + str(Nsample_reliable) + " done in " + str((end-start).seconds) + " seconds!", end ="\r")

### Normalize and Analyze Full experiments

#### Incremental

In [8]:
mm_incremental_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_incremental, "mm","mm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_increment))
sm_incremental_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_incremental, "sm","sm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_increment))

In [8]:
with ProgressBar():
    sm_incremental_norm_ana = statlorenz.Normalize_Analyse(sm_incremental_full_mf, analysis = "analysis", dof=0)
    mm_incremental_norm_ana = statlorenz.Normalize_Analyse(mm_incremental_full_mf, analysis = "analysis", dof=0)

[########################################] | 100% Completed | 14min 32.3s
[########################################] | 100% Completed | 21min 49.7s


In [12]:
sm_incremental_norm_ana.to_netcdf(os.path.join(path_results,"Incremental","sm_incremental_norm_ana.nc"))
mm_incremental_norm_ana.to_netcdf(os.path.join(path_results,"Incremental","mm_incremental_norm_ana.nc"))

### Reliable

In [12]:
mm_reliable_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_reliable, "mm","mm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_reliable))
sm_reliable_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_reliable, "sm","sm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_reliable))

In [15]:
with ProgressBar():
    sm_reliable_norm_ana = statlorenz.Normalize_Analyse(sm_reliable_full_mf, analysis = "analysis", dof=0)
    mm_reliable_norm_ana = statlorenz.Normalize_Analyse(mm_reliable_full_mf, analysis = "analysis", dof=0)

[########################################] | 100% Completed |  3min 15.6s
[########################################] | 100% Completed |  4min 17.6s


In [17]:
sm_reliable_norm_ana.to_netcdf(os.path.join(path_results,"Reliable","sm_reliable_norm_ana.nc"))
mm_reliable_norm_ana.to_netcdf(os.path.join(path_results,"Reliable","mm_reliable_norm_ana.nc"))

### Over

In [10]:
mm_over_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_over, "mm","mm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_over))
sm_over_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_over, "sm","sm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_over))

In [19]:
with ProgressBar():
    sm_over_norm_ana = statlorenz.Normalize_Analyse(sm_over_full_mf, analysis = "analysis", dof=0)
    mm_over_norm_ana = statlorenz.Normalize_Analyse(mm_over_full_mf, analysis = "analysis", dof=0)

[########################################] | 100% Completed |  3min 12.0s
[########################################] | 100% Completed |  4min  2.8s


In [20]:
sm_over_norm_ana.to_netcdf(os.path.join(path_results,"Overconfident","sm_over_norm_ana.nc"))
mm_over_norm_ana.to_netcdf(os.path.join(path_results,"Overconfident","mm_over_norm_ana.nc"))

### Under

In [11]:
mm_under_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_under, "mm","mm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_under))
sm_under_full_mf=xr.open_mfdataset(os.path.join(raw_output_path_under, "sm","sm_???.nc"),concat_dim="sample",combine='nested',parallel=False).assign_coords(sample=np.arange(Nsample_under))

In [22]:
with ProgressBar():
    sm_under_norm_ana = statlorenz.Normalize_Analyse(sm_under_full_mf, analysis = "analysis", dof=0)
    mm_under_norm_ana = statlorenz.Normalize_Analyse(mm_under_full_mf, analysis = "analysis", dof=0)

[########################################] | 100% Completed |  2min 56.9s
[########################################] | 100% Completed |  4min  0.4s


In [23]:
sm_under_norm_ana.to_netcdf(os.path.join(path_results,"Underconfident","sm_under_norm_ana.nc"))
mm_under_norm_ana.to_netcdf(os.path.join(path_results,"Underconfident","mm_under_norm_ana.nc"))

# Calculate Scaife

### Seasonal Mean

In [102]:
path_sm_reliable_norm_scaife       = os.path.join(path_results,"Reliable",       "sm_reliable_norm_scaife.nc")
path_sm_overconfident_norm_scaife  = os.path.join(path_results,"Overconfident",  "sm_over_norm_scaife.nc")
path_sm_underconfident_norm_scaife = os.path.join(path_results,"Underconfident", "sm_under_norm_scaife.nc")

In [25]:
with ProgressBar():
    sm_reliable_norm_scaife        = statlorenz.Normalize_Scaife_Connected(sm_reliable_full_mf, permutations_da=permutations, Nsample=100)
    sm_overconfident_norm_scaife   = statlorenz.Normalize_Scaife_Connected(sm_over_full_mf    , permutations_da=permutations, Nsample=100)
    sm_underconfident_norm_scaife  = statlorenz.Normalize_Scaife_Connected(sm_under_full_mf   , permutations_da=permutations, Nsample=100)
    

[########################################] | 100% Completed |  3hr 53min 33.8s
[########################################] | 100% Completed |  7hr 34min 20.7s
[########################################] | 100% Completed | 11hr  7min  1.1s


In [103]:
sm_reliable_norm_scaife.to_netcdf(path_sm_reliable_norm_scaife)
sm_overconfident_norm_scaife.to_netcdf(path_sm_overconfident_norm_scaife)
sm_underconfident_norm_scaife.to_netcdf(path_sm_underconfident_norm_scaife)

### Monthly Mean

In [130]:
path_mm_reliable_norm_scaife       = os.path.join(path_results,"Reliable",       "mm_reliable_norm_scaife.nc")
path_mm_overconfident_norm_scaife  = os.path.join(path_results,"Overconfident",  "mm_over_norm_scaife.nc")
path_mm_underconfident_norm_scaife = os.path.join(path_results,"Underconfident", "mm_under_norm_scaife.nc")

In [141]:
mm_reliable_norm_scaife        = statlorenz.Normalize_Scaife_Connected(mm_reliable_full_mf.sel({"leadtime in days":[3,6,9,12]}), permutations_da=permutations, Nsample=100)
mm_overconfident_norm_scaife   = statlorenz.Normalize_Scaife_Connected(mm_over_full_mf.sel({"leadtime in days":[3,6,9,12]})    , permutations_da=permutations, Nsample=100)
mm_underconfident_norm_scaife  = statlorenz.Normalize_Scaife_Connected(mm_under_full_mf.sel({"leadtime in days":[3,6,9,12]})   , permutations_da=permutations, Nsample=100)

In [142]:
mm_reliable_norm_scaife.to_netcdf(path_mm_reliable_norm_scaife)
mm_overconfident_norm_scaife.to_netcdf(path_mm_overconfident_norm_scaife)
mm_underconfident_norm_scaife.to_netcdf(path_mm_underconfident_norm_scaife)

## Select Representative

In [21]:
sm_reliable_norm_ana = xr.load_dataset(os.path.join(path_results,"Reliable"      , "sm_reliable_norm_ana.nc"))
sm_under_norm_ana    = xr.load_dataset(os.path.join(path_results,"Underconfident", "sm_under_norm_ana.nc"))
sm_over_norm_ana     = xr.load_dataset(os.path.join(path_results,"Overconfident" , "sm_over_norm_ana.nc"))

In [22]:
sm_reliable_norm_ana_12_x = sm_reliable_norm_ana.sel(dimension="x").sel({"leadtime in days":12})
sm_under_norm_ana_12_x    = sm_under_norm_ana.sel(dimension="x").sel({"leadtime in days":12})
sm_over_norm_ana_12_x     = sm_over_norm_ana.sel(dimension="x").sel({"leadtime in days":12})

In [18]:
reliable_min_distance_index = plotting.calculate_smallest_distance(sm_reliable_norm_ana_12_x["ratio_of_predictable_components_cor"],
                                                                   sm_reliable_norm_ana_12_x["actual_predictability"])
over_min_distance_index = plotting.calculate_smallest_distance(sm_over_norm_ana_12_x["ratio_of_predictable_components_cor"],
                                                                   sm_over_norm_ana_12_x["actual_predictability"])
under_min_distance_index = plotting.calculate_smallest_distance(sm_under_norm_ana_12_x["ratio_of_predictable_components_cor"],
                                                                   sm_under_norm_ana_12_x["actual_predictability"])

In [19]:
reliable_min_distance_index.rename("index").to_netcdf(os.path.join(path_results, "Reliable"      , "sm_reliable_min_distance_index.nc"))
over_min_distance_index.rename("index").to_netcdf(os.path.join(path_results    , "Overconfident" , "sm_over_min_distance_index.nc"    ))
under_min_distance_index.rename("index").to_netcdf(os.path.join(path_results   , "Underconfident", "sm_under_min_distance_index.nc"   ))

/work/uo1075/u301101/conda_envs/jupyterlab_env/lib/python3.8/site-packages/xarray/core/dataarray.py:2266: SerializationWarning: coordinate 'leadtime in days' has a space in its name, which means it cannot be marked as a coordinate on disk and will be saved as a data variable instead
  return dataset.to_netcdf(*args, **kwargs)


# Export Numbers

## Reliable

In [40]:
sm_reliable_norm_ana_12_x_mean_act    = sm_reliable_norm_ana_12_x["actual_predictability"].mean(dim="sample").values
sm_reliable_norm_ana_12_x_meanstd_act = sm_reliable_norm_ana_12_x["actual_predictability"].std(dim="sample").values/np.sqrt(Nsample_reliable)

general.write_number(sm_reliable_norm_ana_12_x_mean_act,    os.path.join(path_numbers,"sm_reliable_mean_act.txt"))
general.write_number(sm_reliable_norm_ana_12_x_meanstd_act, os.path.join(path_numbers,"sm_reliable_meanstd_act.txt")) 

general.latex_export_variable(sm_reliable_norm_ana_12_x_mean_act, 
                              sm_reliable_norm_ana_12_x_meanstd_act, 
                              os.path.join(path_numbers,"sm_reliable_mean_act.txt"), significant_digits=2)

$0.5150\ \pm \ 0.0080$


In [41]:
sm_reliable_norm_ana_12_x_mean_rpc     = sm_reliable_norm_ana_12_x["ratio_of_predictable_components_cor"].mean(dim="sample").values
sm_reliable_norm_ana_12_x_meanstd_rpc  = sm_reliable_norm_ana_12_x["ratio_of_predictable_components_cor"].std(dim="sample").values/np.sqrt(Nsample_reliable)

general.write_number(sm_reliable_norm_ana_12_x_mean_rpc,    os.path.join(path_numbers,"sm_reliable_mean_rpc.txt"))
general.write_number(sm_reliable_norm_ana_12_x_meanstd_rpc, os.path.join(path_numbers,"sm_reliable_meanstd_rpc.txt")) 

general.latex_export_variable(sm_reliable_norm_ana_12_x_mean_rpc, 
                              sm_reliable_norm_ana_12_x_meanstd_rpc, 
                              os.path.join(path_numbers,"sm_reliable_mean_rpc.txt"), significant_digits=2)

$0.985\ \pm \ 0.013$


## Over

In [36]:
sm_over_norm_ana_12_x_mean_act    = sm_over_norm_ana_12_x["actual_predictability"].mean(dim="sample").values
sm_over_norm_ana_12_x_meanstd_act = sm_over_norm_ana_12_x["actual_predictability"].std(dim="sample").values/np.sqrt(Nsample_over)


general.write_number(sm_over_norm_ana_12_x_mean_act,    os.path.join(path_numbers,"sm_over_mean_act.txt"))
general.write_number(sm_over_norm_ana_12_x_meanstd_act, os.path.join(path_numbers,"sm_over_meanstd_act.txt")) 

general.latex_export_variable(sm_over_norm_ana_12_x_mean_act, 
                              sm_over_norm_ana_12_x_meanstd_act, 
                              os.path.join(path_numbers,"sm_over_mean_act_latex.txt"), significant_digits=2)

$0.4319\ \pm \ 0.0091$


In [37]:
sm_over_norm_ana_12_x_mean_rpc    = sm_over_norm_ana_12_x["ratio_of_predictable_components_cor"].mean(dim="sample").values
sm_over_norm_ana_12_x_meanstd_rpc = sm_over_norm_ana_12_x["ratio_of_predictable_components_cor"].std(dim="sample").values/np.sqrt(Nsample_over)

general.write_number(sm_over_norm_ana_12_x_mean_rpc,    os.path.join(path_numbers,"sm_over_mean_rpc.txt"))
general.write_number(sm_over_norm_ana_12_x_meanstd_rpc, os.path.join(path_numbers,"sm_over_meanstd_rpc.txt")) 

general.latex_export_variable(sm_over_norm_ana_12_x_mean_rpc, 
                              sm_over_norm_ana_12_x_meanstd_rpc, 
                              os.path.join(path_numbers,"sm_over_mean_rpc.txt"), significant_digits=2)

$0.584\ \pm \ 0.012$


## Under

In [38]:
sm_under_norm_ana_12_x_mean_act    = sm_under_norm_ana_12_x["actual_predictability"].mean(dim="sample").values
sm_under_norm_ana_12_x_meanstd_act = sm_under_norm_ana_12_x["actual_predictability"].std(dim="sample").values/np.sqrt(Nsample_under)

general.write_number(sm_under_norm_ana_12_x_mean_act,    os.path.join(path_numbers,"sm_under_mean_act.txt"))
general.write_number(sm_under_norm_ana_12_x_meanstd_act, os.path.join(path_numbers,"sm_under_meanstd_act.txt")) 

general.latex_export_variable(sm_under_norm_ana_12_x_mean_act, 
                              sm_under_norm_ana_12_x_meanstd_act, 
                              os.path.join(path_numbers,"sm_over_mean_act.txt"), significant_digits=2)

$0.3152\ \pm \ 0.0088$


In [39]:
sm_under_norm_ana_12_x_mean_rpc    = sm_under_norm_ana_12_x["ratio_of_predictable_components_cor"].mean(dim="sample").values
sm_under_norm_ana_12_x_meanstd_rpc = sm_under_norm_ana_12_x["ratio_of_predictable_components_cor"].std(dim="sample").values/np.sqrt(Nsample_under)

general.write_number(sm_under_norm_ana_12_x_mean_rpc,    os.path.join(path_numbers,"sm_under_mean_rpc.txt"))
general.write_number(sm_under_norm_ana_12_x_meanstd_rpc, os.path.join(path_numbers,"sm_under_meanstd_rpc.txt")) 

general.latex_export_variable(sm_under_norm_ana_12_x_mean_rpc, 
                              sm_under_norm_ana_12_x_meanstd_rpc, 
                              os.path.join(path_numbers,"sm_under_mean_rpc.txt"), significant_digits=2)

$1.369\ \pm \ 0.038$
